In [3]:
import sys
sys.path.insert(0, "./lib/")

import VOC2012DataProvider
import VOC2012_npz_files_writter
import tensorflow as tf
import os.path

/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.23) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
#This cell reads VOC 2012 dataset and save them in .npz files for future.
#The process of reading data and put them in prper format is time consuming so they are stored in a file.

xml_path = "../VOC2012/Annotations/*.xml"
destination = "../testdata/"

#It splits dataset to 80% for training and 20% validation.
if not (os.path.isfile(destination+"test_input.npz") or os.path.isfile(destination+"test_target.npz")):
    VOC2012_npz_files_writter.writting_files(xml_path, destination, percentage=0)
    print("Files are ready!!!")
else:
    print("TFrecords are already prepared!!!")

Reading dataset is started. Please wait it might take several minutes to create .npz files ...
train1_target.npz and train1_input.npz are being written ...
Files are written. It's done. PLease keep waiting ...
train2_target.npz and train2_input.npz are being written ...
Files are written. It's done. PLease keep waiting ...
train3_target.npz and train3_input.npz are being written ...
Files are written. It's done.
test_target.npz and test_input.npz are being written ...
Files are written. It's done.
Successfully converted dataset to .npz files.
training dataset: # 
13700
test dataset: # 
3425
TFrecords are ready!!!


In [4]:
inputs = VOC2012DataProvider.PascalDataProvider('train', batch_size=50)

In [8]:
inputs.

In [3]:
# Convolutional Layer 1.# Convo 
filter_size1 = 7          # Convolution filters are 5 x 5 pixels.
num_filters1 = 96         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 7          # Convolution filters are 5 x 5 pixels.
num_filters2 = 265        # There are 36 of these filters.

# Convolutional Layer 3.
filter_size2 = 3          # Convolution filters are 5 x 5 pixels.
num_filters2 = 384        # There are 36 of these filters.

# Convolutional Layer 2.
filter_size2 = 3          # Convolution filters are 5 x 5 pixels.
num_filters2 = 384        # There are 36 of these filters.


# Fully-connected layer.
fc_size = 4096           # Number of neurons in fully-connected layer.
fc_size = 1024           # Number of neurons in fully-connected layer.
fc_size = 1024           # Number of neurons in fully-connected layer.
#fc_size = 9           # Number of neurons in fully-connected layer.

In [4]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 224

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 3

# Number of classes, one class for each of 10 digits.
num_classes = 9

In [5]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [6]:
def new_conv_layer(input,              # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   filter_stride,
                   use_pooling,   # Use max-pooling.
                   pooling_size = None,
                   pooling_stride = None):  

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, filter_stride, filter_stride, 1], #strides=[batch, height, width, channels]
                         padding='SAME') # 'SAME' means zero-padding

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, pooling_size, pooling_size, 1],
                               strides=[1, pooling_stride, pooling_stride, 1], #strides=[batch, height, width, channels]
                               padding='SAME') # 'SAME' means zero-padding

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

In [ ]:
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

In [ ]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [ ]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=3,
                   filter_size=7,
                   num_filters=96,
                   filter_stride = 2,
                   use_pooling=True,   # Use max-pooling.
                   pooling_size = 3,
                   pooling_stride = 2)

layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=96,
                   filter_size=5,
                   num_filters=256,
                   filter_stride = 2,
                   use_pooling=True,   # Use max-pooling.
                   pooling_size = 3,
                   pooling_stride = 1)

layer_conv3, weights_conv3 = \
    new_conv_layer(input=layer_conv2,
                   num_input_channels=256,
                   filter_size=3,
                   num_filters=384,
                   filter_stride = 1,
                   use_pooling=False,   # Use max-pooling.
                  )

layer_conv4, weights_conv4 = \
    new_conv_layer(input=layer_conv3,
                   num_input_channels=384,
                   filter_size=3,
                   num_filters=384,
                   filter_stride = 1,
                   use_pooling=False,   # Use max-pooling.
                  )

layer_conv5, weights_conv5 = \
    new_conv_layer(input=layer_conv4,
                   num_input_channels=384,
                   filter_size=0,
                   num_filters=0,
                   filter_stride = 0,
                   use_pooling=True,   # Use max-pooling.
                   pooling_size = 3,
                   pooling_stride = 2)    

In [ ]:
layer_flat, num_features = flatten_layer(layer_conv5)

layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=1024,
                         use_relu=True)

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=1024,
                         num_outputs=20,
                         use_relu=False)

In [ ]:
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, axis=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [ ]:
optimizeroptimize  = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for e in range(10):
    
    running_error = 0.
    running_accuracy = 0.
    
    record_iterator = tf.python_io.tf_record_iterator(path="../Tfdata/mytest.record")

    for string_record in record_iterator:
    
        example = tf.train.Example()
        
        example.ParseFromString(string_record)
    
    
    for input_batch, target_batch in train_data:
        _, batch_error, batch_acc = sess.run(
            [optimizer, cost, accuracy], 
            feed_dict={x: input_batch, y_true: target_batch})
        running_error += batch_error
        running_accuracy += batch_acc
    running_error /= train_data.num_batches
    running_accuracy /= train_data.num_batches
    print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f}'
          .format(e + 1, running_error, running_accuracy))
    if (e + 1) % 5 == 0:
        valid_error = 0.
        valid_accuracy = 0.
        for input_batch, target_batch in valid_data:
            batch_error, batch_acc = sess.run([cost, accuracy],feed_dict={x: input_batch, y_true: target_batch})
            valid_error += batch_error
            valid_accuracy += batch_acc
        valid_error /= valid_data.num_batches
        valid_accuracy /= valid_data.num_batches
        print('                 err(valid)={0:.2f} acc(valid)={1:.2f}'
               .format(valid_error, valid_accuracy))

link to load data from pascal VOC: https://gist.github.com/saghiralfasly/ee642af0616461145a9a82d7317fb1d6

http://machinelearninguru.com/deep_learning/tensorflow/basics/tfrecord/tfrecord.html

Computing IoU: https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
https://angusg.com/writing/2016/12/28/optimizing-iou-semantic-segmentation.html